In [1]:
# import modules
import pandas as pd
import numpy as np
import subprocess
import re

In [7]:
# optuna_study.db에서 best parameter set 불러오기
import sqlite3
import optuna

# optuna db 연결
storage_name = "sqlite:///optuna_study.db" 
study = optuna.load_study(study_name="parameter_optimization", storage=storage_name)

# best trial의 parameters 가져오기
best_params = study.best_trial.params

# best parameters 출력
print(f"Best parameters from Optuna study:")
for param_name, value in best_params.items():
    print(f"{param_name}: {value}")


KeyError: 'Record does not exist.'

In [ ]:
# 특정 trial의 parameter를 이용해서 kinet_opt를 만드는 함수
def create_kinet_opt_from_trial(trial_number):
    """
    특정 trial 번호의 parameter를 이용해서 kinet_opt.inp 파일을 생성하는 함수
    
    Args:
        trial_number (int): 사용할 trial 번호
    """
    # 특정 trial의 parameters 가져오기
    trial = study.get_trials()[trial_number]
    trial_params = trial.params
    
    print(f"Trial {trial_number}의 parameters:")
    for param_name, value in trial_params.items():
        print(f"{param_name}: {value}")
    
    # kinet_ori.inp 파일 읽기
    with open('kinet_ori.inp', 'r') as f:
        content = f.readlines()
    
    # 파라미터 선언 위치 찾기
    param_start = -1
    for i, line in enumerate(content):
        if "$ double precision, parameter :: f" in line:
            param_start = i
            break
    
    if param_start != -1:
        # 새로운 파라미터 값들 생성
        new_params = []
        for param_name, value in trial_params.items():
            fortran_value = f"{value:.4e}".replace('e', 'd')
            new_params.append(f"$ double precision, parameter :: {param_name} = {fortran_value}\n")
        
        # kinet_trial_{trial_number}.inp 파일 생성
        output_filename = f'kinet_trial_{trial_number}.inp'
        with open(output_filename, 'w') as f:
            f.writelines(content[:param_start])  # 파라미터 선언 이전 부분
            f.writelines(new_params)  # 새로운 파라미터 값들
            f.writelines(content[param_start + len(trial_params):])  # 파라미터 선언 이후 부분
        
        print(f"\n{output_filename} 파일이 생성되었습니다.")
        return output_filename
    else:
        print("파라미터 선언 부분을 찾을 수 없습니다.")
        return None

# 사용 예시
# create_kinet_opt_from_trial(0)  # 첫 번째 trial의 parameter 사용
# create_kinet_opt_from_trial(5)  # 6번째 trial의 parameter 사용

# 모든 trial의 parameter로 파일들을 생성하는 함수
def create_all_trial_files():
    """
    모든 trial의 parameter를 이용해서 각각의 kinet 파일들을 생성하는 함수
    """
    trials = study.get_trials()
    created_files = []
    
    for i, trial in enumerate(trials):
        if trial.state == optuna.trial.TrialState.COMPLETE:  # 완료된 trial만 처리
            filename = create_kinet_opt_from_trial(i)
            if filename:
                created_files.append(filename)
    
    print(f"\n총 {len(created_files)}개의 파일이 생성되었습니다.")
    return created_files

# 특정 trial의 objective value도 확인하는 함수
def get_trial_info(trial_number):
    """
    특정 trial의 상세 정보를 출력하는 함수
    
    Args:
        trial_number (int): 확인할 trial 번호
    """
    trial = study.get_trials()[trial_number]
    print(f"Trial {trial_number} 정보:")
    print(f"  State: {trial.state}")
    print(f"  Objective Value: {trial.value}")
    print(f"  Parameters:")
    for param_name, value in trial.params.items():
        print(f"    {param_name}: {value}")
    print(f"  Duration: {trial.duration}")
    print(f"  Start Time: {trial.start}")
    print(f"  Complete Time: {trial.complete}")



In [3]:
# best parameter set을 double fortran 형식으로 출력
print("\nBest parameters in Fortran double precision format:")
for param_name, value in best_params.items():
    fortran_value = f"{value:.4e}".replace('e', 'd')
    print(f"$ double precision, parameter :: {param_name} = {fortran_value}")

# kinet_opt.inp 파일 생성
with open('kinet_ori.inp', 'r') as f:
    content = f.readlines()

# 파라미터 선언 위치 찾기
param_start = -1
for i, line in enumerate(content):
    if "$ double precision, parameter :: f" in line:
        param_start = i
        break

if param_start != -1:
    # 기존 파라미터 선언 부분을 새로운 값으로 교체
    new_params = []
    for param_name, value in best_params.items():
        fortran_value = f"{value:.4e}".replace('e', 'd')
        new_params.append(f"$ double precision, parameter :: {param_name} = {fortran_value}\n")
    
    # 새로운 내용으로 파일 작성
    with open('kinet_opt.inp', 'w') as f:
        f.writelines(content[:param_start])  # 파라미터 선언 이전 부분
        f.writelines(new_params)  # 새로운 파라미터 값들
        f.writelines(content[param_start + len(best_params):])  # 파라미터 선언 이후 부분
    
    print("\nkinet_opt.inp 파일이 생성되었습니다.")
else:
    print("파라미터 선언 부분을 찾을 수 없습니다.")




Best parameters in Fortran double precision format:
$ double precision, parameter :: f0 = 1.5531d-06
$ double precision, parameter :: f1 = 3.0782d-11
$ double precision, parameter :: f2 = 1.2242d+07
$ double precision, parameter :: f3 = 8.2873d+06
$ double precision, parameter :: f4 = 1.8951d+00
$ double precision, parameter :: f5 = 4.2995d+03
$ double precision, parameter :: f6 = 5.3434d+00
$ double precision, parameter :: f7 = 2.1654d+06
$ double precision, parameter :: f8 = 2.3360d+01
$ double precision, parameter :: f9 = 1.6726d+02
$ double precision, parameter :: f10 = 2.9871d-02
$ double precision, parameter :: f11 = 3.7054d+02
$ double precision, parameter :: f12 = 1.1273d-04
$ double precision, parameter :: f13 = 5.7060d-01
$ double precision, parameter :: f14 = 6.6972d-02
$ double precision, parameter :: f15 = 4.4730d-02
$ double precision, parameter :: f16 = 1.3608d+11
$ double precision, parameter :: f17 = 9.4624d-01
$ double precision, parameter :: f18 = 8.8196d+16
$ doubl